In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

^C


In [3]:
pip install -U scikit-learn scipy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mediapipe as mp
import cv2 

In [2]:
# Drawing helpers and Mediapipe Solutions
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

## 1. Dibujar landmarks con mediapipe 

In [79]:
# Inicializa la captura de video desde la webcam
cap = cv2.VideoCapture(0)

# Inicia el modelo Holistic de mediapipe para la detección de landmarks
# Los argumentos  min_detection_confidence y min_tracking_confidence  establecen los umbrales de confianza mínimos requeridos 
#para la detección y el seguimiento de landmarks en una imagen.
#min_detection_confidence: Este argumento establece el umbral de confianza mínimo necesario para que el modelo Holistic detecte un landmark en la imagen. Un valor más alto significa que el modelo solo detectará landmarks con una confianza muy alta, mientras que un valor más bajo permitirá detectar landmarks con una confianza más baja.
#min_tracking_confidence: Este argumento establece el umbral de confianza mínimo para el seguimiento de landmarks en imágenes sucesivas. Una vez que un landmark ha sido detectado en una imagen con una confianza superior a este umbral, el modelo intentará seguir ese landmark en imágenes posteriores. Un valor más alto aquí significa que el seguimiento se realizará solo para landmarks muy confiables, mientras que un valor más bajo permitirá un seguimiento incluso para landmarks menos confiables.
# Factore: buena iluminación, fondo simple, si se requiere precision : usar valores mas alto
# Un valor muy alto de confianza puede conducir a una menor cantidad de landmarks detectados y seguidos, lo que puede ser aceptable en algunos casos pero limitante en otros donde se necesite información más detallada.
#min_detection_confidence: deteccion
#min_tracking_confidence: seguimiento
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.6) as holistic: #probar: min_detection_confidence=0.7-0.9, min_tracking_confidence=0.5-0.8   0.5 y 0.8
    while cap.isOpened():  # Bucle principal para procesar cada fotograma de video
        ret, frame = cap.read()  # Lee un fotograma de la webcam
        
        # Recolor Feed
        # Convierte el fotograma de BGR a RGB para el procesamiento de mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False   # Marca la imagen como no modificable   
        
        # Make Detections
        # Procesa la imagen utilizando el modelo Holistic para detectar landmarks
        results = holistic.process(image)
        # print(results.face_landmarks) # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Muestra la imagen con los landmarks dibujados en una ventana
        cv2.imshow('Rosa Webcam', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [80]:
results.pose_landmarks

landmark {
  x: 0.2459341287612915
  y: 0.4354501962661743
  z: -0.49546346068382263
  visibility: 0.9991111159324646
}
landmark {
  x: 0.27558434009552
  y: 0.3823970556259155
  z: -0.5393932461738586
  visibility: 0.9984203577041626
}
landmark {
  x: 0.2897672653198242
  y: 0.3808986246585846
  z: -0.5397096872329712
  visibility: 0.9986534118652344
}
landmark {
  x: 0.30426809191703796
  y: 0.3781900107860565
  z: -0.5396732687950134
  visibility: 0.9983720183372498
}
landmark {
  x: 0.2495085746049881
  y: 0.3821074366569519
  z: -0.4532240927219391
  visibility: 0.998242199420929
}
landmark {
  x: 0.24449646472930908
  y: 0.3817051947116852
  z: -0.4532557427883148
  visibility: 0.9984951615333557
}
landmark {
  x: 0.24064302444458008
  y: 0.38104259967803955
  z: -0.45389169454574585
  visibility: 0.9983667135238647
}
landmark {
  x: 0.3648405969142914
  y: 0.3958626091480255
  z: -0.5214162468910217
  visibility: 0.9987645745277405
}
landmark {
  x: 0.2874719500541687
  y: 0.394

In [133]:
results.face_landmarks.landmark[0].visibility # visibility muestra si el landmark si se muestra en la pantalla o no
# para los landmarks de la cara siempre sera cero

0.0

## 2. Capturar Landmarks & Exportar CSV

In [134]:
import csv
import os
import numpy as np

In [135]:
num_coords = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_coords #501

501

In [136]:
#Create header table | class | x | y | z | visibility |
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [137]:
len(landmarks) #2005

2005

In [138]:
#Export header CSV
with open('temporalidad_1_0.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## 3. Tomar coordenadas de señas

In [141]:
#Save coord poses
class_name="aveces"

#Detectar puntos de referencia
cap = cv2.VideoCapture(0)  #0 o 2

# Inicializar el tiempo de inicio
inicio_tiempo = cv2.getTickCount()

# Initiate holistic model
#min_detection_confidence: deteccion
#min_tracking_confidence: seguimiento
with mp_holistic.Holistic(min_detection_confidence=0.9, min_tracking_confidence=0.8) as holistic: #probar: min_detection_confidence=0.7-0.9, min_tracking_confidence=0.5-0.8

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )

        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates, da todos los landmarks y los almacena en una matriz
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            # Concate rows
            row = pose_row+face_row

            # Append class name
            row.insert(0, class_name)

            # Export to CSV
            with open('temporalidad_1_0.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass

        ##cv2.imshow('Rosa Webcam', image)
        ##if cv2.waitKey(10) & 0xFF == ord('q'):
        ##    break
        # Calcular el tiempo transcurrido
        tiempo_transcurrido = (cv2.getTickCount() - inicio_tiempo) / cv2.getTickFrequency()

        # Mostrar la imagen y salir si pasa el tiempo límite o se presiona 'q'
        cv2.imshow('Rosa Webcam', image)
        if tiempo_transcurrido >= 60 or cv2.waitKey(10) & 0xFF == ord('q'):
            break
        

cap.release()
cv2.destroyAllWindows()


## 4. Entrenar modelo usando Scikit Learn

In [142]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [143]:
df = pd.read_csv('temporalidad_1_0.csv')

In [144]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,mes,0.549135,0.394194,-0.809048,0.999992,0.579765,0.348766,-0.752098,0.999985,0.595620,...,0.001607,0.0,0.600875,0.335847,0.014170,0.0,0.605487,0.332054,0.014272,0.0
1,mes,0.546055,0.395098,-1.060882,0.999991,0.576010,0.349668,-0.995259,0.999983,0.590626,...,0.001446,0.0,0.598345,0.339420,0.014053,0.0,0.602958,0.335890,0.014159,0.0
2,mes,0.542215,0.396934,-0.999211,0.999990,0.570538,0.351776,-0.935079,0.999982,0.585182,...,0.001247,0.0,0.592184,0.343229,0.013815,0.0,0.596732,0.339478,0.013969,0.0
3,mes,0.538879,0.398038,-0.817963,0.999988,0.565235,0.353356,-0.752814,0.999978,0.580477,...,0.001401,0.0,0.587346,0.348871,0.013566,0.0,0.591874,0.345217,0.013690,0.0
4,mes,0.536219,0.401987,-0.740743,0.999984,0.561953,0.357833,-0.679093,0.999971,0.577154,...,0.000645,0.0,0.583073,0.355737,0.011841,0.0,0.587611,0.352066,0.011862,0.0


In [145]:
df[df['class']=='anio']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
415,anio,0.537167,0.379919,-0.727094,0.999994,0.569766,0.327761,-0.675342,0.999989,0.587081,...,0.002245,0.0,0.589399,0.316334,0.010716,0.0,0.594814,0.311495,0.010657,0.0
416,anio,0.538806,0.379869,-1.088912,0.999988,0.570749,0.327520,-1.013447,0.999979,0.587151,...,0.001376,0.0,0.591470,0.316294,0.010541,0.0,0.596641,0.311898,0.010466,0.0
417,anio,0.537832,0.380505,-1.123347,0.999983,0.570560,0.327515,-1.053539,0.999974,0.586938,...,0.001039,0.0,0.589929,0.320670,0.009411,0.0,0.595237,0.315857,0.009166,0.0
418,anio,0.537042,0.380739,-1.173109,0.999978,0.570454,0.327509,-1.100974,0.999967,0.586798,...,0.000174,0.0,0.587900,0.321684,0.007567,0.0,0.593237,0.316653,0.007254,0.0
419,anio,0.537149,0.381378,-1.129792,0.999975,0.571131,0.327530,-1.057510,0.999964,0.587072,...,-0.000193,0.0,0.591049,0.327298,0.007726,0.0,0.596778,0.320559,0.007746,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,anio,0.540826,0.418476,-0.787728,0.999902,0.562513,0.374420,-0.714568,0.999735,0.574644,...,0.004085,0.0,0.583972,0.362799,0.019421,0.0,0.588139,0.359515,0.019948,0.0
824,anio,0.540816,0.418981,-0.821835,0.999907,0.562678,0.374692,-0.747167,0.999749,0.574945,...,0.004282,0.0,0.584778,0.362764,0.019610,0.0,0.588980,0.359409,0.020158,0.0
825,anio,0.540807,0.418862,-0.774413,0.999913,0.562752,0.374679,-0.695256,0.999764,0.575028,...,0.005107,0.0,0.584656,0.365388,0.019967,0.0,0.589132,0.360721,0.020684,0.0
826,anio,0.540889,0.418715,-0.803152,0.999916,0.562976,0.374678,-0.724844,0.999772,0.575379,...,0.004595,0.0,0.584939,0.363001,0.020209,0.0,0.589117,0.359709,0.020751,0.0


In [146]:
X = df.drop('class', axis=1) # coordenadas
y = df['class'] # clase

In [147]:
y #X = coords , y = class

0          mes
1          mes
2          mes
3          mes
4          mes
         ...  
1257    aveces
1258    aveces
1259    aveces
1260    aveces
1261    aveces
Name: class, Length: 1262, dtype: object

train_test_split(): dividide los datos en conjuntos de entrenamiento y prueba.
- test_size: Es el tamaño que se desea para el conjunto de prueba. Se establece en 0.3, lo que significa que el 30% de los datos se asignarán al conjunto de prueba, y el 70% restante se utilizará para entrenamiento.
- random_state: Este parámetro se utiliza para inicializar el generador de números aleatorios interno, lo que garantiza que la división de los datos sea reproducible

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [149]:
X_train

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
1071,0.531715,0.408507,-0.618889,0.999999,0.558529,0.359588,-0.564094,0.999998,0.571593,0.361934,...,0.002819,0.0,0.582799,0.346580,0.013658,0.0,0.587393,0.343068,0.013838,0.0
480,0.544767,0.425135,-0.819661,0.999923,0.570177,0.379395,-0.745749,0.999802,0.582672,0.380828,...,0.001898,0.0,0.587786,0.370268,0.012536,0.0,0.592594,0.365621,0.012824,0.0
755,0.545779,0.425959,-0.960496,0.999931,0.568995,0.382697,-0.896343,0.999815,0.583019,0.385348,...,0.003339,0.0,0.584947,0.361260,0.018827,0.0,0.588935,0.358371,0.019233,0.0
463,0.532203,0.395383,-0.734580,0.999941,0.549578,0.352531,-0.647906,0.999816,0.561977,0.353974,...,0.005607,0.0,0.572163,0.359535,0.017878,0.0,0.576444,0.354848,0.018105,0.0
1220,0.539375,0.406473,-0.767139,0.999997,0.566482,0.360515,-0.695487,0.999993,0.579690,0.363167,...,0.001953,0.0,0.587790,0.350942,0.011578,0.0,0.592365,0.347192,0.011656,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,0.544871,0.403599,-0.954051,0.999895,0.562191,0.366016,-0.873285,0.999703,0.573294,0.367486,...,0.007831,0.0,0.583905,0.363821,0.025952,0.0,0.588535,0.359029,0.026828,0.0
1228,0.538455,0.406140,-0.865450,0.999996,0.565894,0.360500,-0.794276,0.999992,0.579450,0.363064,...,0.001946,0.0,0.587157,0.352672,0.011117,0.0,0.591865,0.348543,0.011231,0.0
1077,0.527681,0.406673,-0.637033,1.000000,0.554090,0.358076,-0.577166,0.999999,0.567700,0.359977,...,0.003706,0.0,0.574409,0.350083,0.014133,0.0,0.579326,0.344993,0.014549,0.0
723,0.556669,0.400688,-1.033170,0.999843,0.572270,0.361469,-0.947566,0.999589,0.583556,0.363792,...,0.008003,0.0,0.584181,0.361442,0.025804,0.0,0.588740,0.357509,0.026512,0.0


In [150]:
df #marco de datos completo

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,mes,0.549135,0.394194,-0.809048,0.999992,0.579765,0.348766,-0.752098,0.999985,0.595620,...,0.001607,0.0,0.600875,0.335847,0.014170,0.0,0.605487,0.332054,0.014272,0.0
1,mes,0.546055,0.395098,-1.060882,0.999991,0.576010,0.349668,-0.995259,0.999983,0.590626,...,0.001446,0.0,0.598345,0.339420,0.014053,0.0,0.602958,0.335890,0.014159,0.0
2,mes,0.542215,0.396934,-0.999211,0.999990,0.570538,0.351776,-0.935079,0.999982,0.585182,...,0.001247,0.0,0.592184,0.343229,0.013815,0.0,0.596732,0.339478,0.013969,0.0
3,mes,0.538879,0.398038,-0.817963,0.999988,0.565235,0.353356,-0.752814,0.999978,0.580477,...,0.001401,0.0,0.587346,0.348871,0.013566,0.0,0.591874,0.345217,0.013690,0.0
4,mes,0.536219,0.401987,-0.740743,0.999984,0.561953,0.357833,-0.679093,0.999971,0.577154,...,0.000645,0.0,0.583073,0.355737,0.011841,0.0,0.587611,0.352066,0.011862,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,aveces,0.539636,0.408666,-0.746045,0.999998,0.567139,0.361573,-0.679805,0.999996,0.580581,...,0.001693,0.0,0.588285,0.351952,0.011262,0.0,0.592898,0.348152,0.011333,0.0
1258,aveces,0.539683,0.408355,-0.682429,0.999998,0.567101,0.361202,-0.617916,0.999996,0.580461,...,0.001860,0.0,0.589156,0.351018,0.011797,0.0,0.593730,0.347322,0.011862,0.0
1259,aveces,0.539697,0.406225,-0.688693,0.999998,0.567089,0.359371,-0.620507,0.999995,0.580446,...,0.001832,0.0,0.589122,0.350791,0.011516,0.0,0.593706,0.347040,0.011572,0.0
1260,aveces,0.539707,0.405756,-0.715585,0.999997,0.567082,0.358897,-0.655038,0.999995,0.580450,...,0.001758,0.0,0.588231,0.351193,0.011012,0.0,0.592886,0.347285,0.011065,0.0


## Modelo de clasificación de aprendizaje automático

In [151]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [153]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [154]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [155]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [156]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] =  model
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [157]:
fit_models['rc'].predict(X_test) 

array(['aveces', 'aveces', 'mes', 'aveces', 'anio', 'anio', 'mes',
       'aveces', 'anio', 'aveces', 'anio', 'aveces', 'aveces', 'anio',
       'aveces', 'anio', 'anio', 'mes', 'aveces', 'aveces', 'mes', 'mes',
       'anio', 'aveces', 'aveces', 'mes', 'mes', 'aveces', 'aveces',
       'anio', 'mes', 'aveces', 'anio', 'anio', 'anio', 'anio', 'aveces',
       'aveces', 'anio', 'mes', 'mes', 'mes', 'anio', 'anio', 'aveces',
       'aveces', 'anio', 'aveces', 'anio', 'mes', 'anio', 'anio', 'mes',
       'anio', 'aveces', 'anio', 'anio', 'aveces', 'aveces', 'anio',
       'aveces', 'mes', 'aveces', 'mes', 'aveces', 'aveces', 'anio',
       'aveces', 'aveces', 'aveces', 'mes', 'mes', 'aveces', 'mes',
       'aveces', 'aveces', 'anio', 'anio', 'mes', 'anio', 'anio', 'anio',
       'aveces', 'anio', 'aveces', 'aveces', 'mes', 'mes', 'mes',
       'aveces', 'anio', 'anio', 'aveces', 'aveces', 'aveces', 'mes',
       'mes', 'mes', 'anio', 'mes', 'mes', 'aveces', 'mes', 'aveces',
       'aveces

In [158]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [159]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test) ## yhat almacena las predicciones reales  
    print(algo, accuracy_score(y_test, yhat)) #imprimimos la precision, 

lr 0.9973614775725593
rc 1.0
rf 0.9973614775725593
gb 1.0


In [160]:
fit_models['rc'].predict(X_test)

array(['aveces', 'aveces', 'mes', 'aveces', 'anio', 'anio', 'mes',
       'aveces', 'anio', 'aveces', 'anio', 'aveces', 'aveces', 'anio',
       'aveces', 'anio', 'anio', 'mes', 'aveces', 'aveces', 'mes', 'mes',
       'anio', 'aveces', 'aveces', 'mes', 'mes', 'aveces', 'aveces',
       'anio', 'mes', 'aveces', 'anio', 'anio', 'anio', 'anio', 'aveces',
       'aveces', 'anio', 'mes', 'mes', 'mes', 'anio', 'anio', 'aveces',
       'aveces', 'anio', 'aveces', 'anio', 'mes', 'anio', 'anio', 'mes',
       'anio', 'aveces', 'anio', 'anio', 'aveces', 'aveces', 'anio',
       'aveces', 'mes', 'aveces', 'mes', 'aveces', 'aveces', 'anio',
       'aveces', 'aveces', 'aveces', 'mes', 'mes', 'aveces', 'mes',
       'aveces', 'aveces', 'anio', 'anio', 'mes', 'anio', 'anio', 'anio',
       'aveces', 'anio', 'aveces', 'aveces', 'mes', 'mes', 'mes',
       'aveces', 'anio', 'anio', 'aveces', 'aveces', 'aveces', 'mes',
       'mes', 'mes', 'anio', 'mes', 'mes', 'aveces', 'mes', 'aveces',
       'aveces

In [161]:
y_test

1121    aveces
1000    aveces
373        mes
1013    aveces
791       anio
         ...  
4          mes
114        mes
1253    aveces
171        mes
858     aveces
Name: class, Length: 379, dtype: object

In [162]:
# Guardar el modelo ya entrenado
with open('temporalidad_1_0.pkl', 'wb') as f: 
    pickle.dump(fit_models['rf'], f) 

## Cargar el modelo ya entrenado

In [163]:
with open('temporalidad_1_0.pkl', 'rb') as f:
    model = pickle.load(f)

In [164]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [166]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.6) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, ''
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, ''
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Rosa Webcam', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


anio [0.61 0.32 0.07]


c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


anio [0.6  0.32 0.08]
anio [0.61 0.32 0.07]


c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

anio [0.79 0.12 0.09]
anio [0.82 0.1  0.08]


c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


anio [0.78 0.12 0.1 ]


c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\jpc\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


anio [0.67 0.11 0.22]
